# ACDC 2022 - ERA5 Exploration Jupyter Notebook
#### M. M. Lague; Aug. 2022

Loads and plots some ERA5 data on a laptop

-----

Load a bunch of packages that we might use

In [1]:
%matplotlib inline
import sys
sys.path.append('./scripts')

# netcdf/numpy/xarray
import numpy as np
import netCDF4 as nc
import numpy.matlib
import datetime
import xarray as xr
from scipy import interpolate
from numpy import ma
from scipy import stats
import scipy.io as sio
import pickle as pickle
from sklearn import linear_model
import numpy.ma as ma
import matplotlib.patches as mpatches

import scipy as sp
import pandas as pd

import time

from copy import copy 

#from joblib import Parallel, delayed
import multiprocessing
import dask
import dask.bag as db

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker

from matplotlib.ticker import FormatStrFormatter

from matplotlib import gridspec

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point



# OS interaction
import os
import sys
import cftime
import datetime

import glob

import xesmf as xe
# import esmf as xe
import ESMF



from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)


# from windspharm.standard import VectorWind
# import spharm

----

## Link external harddrive
At least for me, my local jupyterlab session won't access data on an external drive unless I create a symbolic link from somewhere the session *can* access - I'm putting it in the same place as this notebook.

`ln -s /Volumes/OneTouch/ACDC/ERA5_ACDC/ /Users/mlague/Documents/ACDC/ERA5_ACDC`


----

### Load a bunch of ERA5 data from an external drive


In [3]:
era5_path_fine = '/Users/mlague/Documents/ACDC/ERA5_ACDC/' #variables/raw/
era5_path_coarse = '/Users/mlague/Documents/ACDC/ERA5_ACDC_1deg/'

list what surface (or non-vertical) variables we'd like, and what vertical variables we'd like. I do this separately because the vertical ones are a lot bigger, so it is a quick thing to comment out if I don't want to load vertically resolved stuff.

The ERA5 data is pretty high resolution, so the files are ALL kind of big. Xarray and dask should "lazy" load the data we ask for - it won't actually read it into memory until we try and do some math or plotting with it. 

In [4]:
varlist = next(os.walk(era5_path_fine + 'variables/raw/'))[1]

In [5]:
varlist

['2m_dewpoint_temperature',
 '2m_temperature',
 'specific_humidity',
 'relative_humidity',
 'geopotential',
 'surface_net_solar_radiation',
 'surface_latent_heat_flux',
 'mean_sea_level_pressure',
 'leaf_area_index_high_vegetation',
 'leaf_area_index_low_vegetation',
 'medium_cloud_cover',
 'sea_surface_temperature',
 'high_cloud_cover',
 'low_cloud_cover',
 'skin_temperature',
 'u_component_of_wind',
 'v_component_of_wind',
 'temperature',
 'surface_solar_radiation_downwards',
 'surface_net_thermal_radiation',
 'surface_sensible_heat_flux',
 'top_net_solar_radiation',
 'top_net_thermal_radiation',
 'total_cloud_cover',
 'surface_pressure',
 'surface_thermal_radiation_downwards',
 'toa_incident_solar_radiation',
 'total_precipitation',
 'vertical_integral_of_northward_total_energy_flux',
 'vertically_integrated_moisture_divergence',
 'vertical_integral_of_temperature',
 'vertical_integral_of_thermal_energy',
 'vertical_integral_of_total_energy']

load a 1 degree file

In [6]:
ds_1deg = xr.open_dataset(era5_path_coarse + 'hurs_1pctCO2-bgc_high_mean_interp.nc')
ds_25deg = xr.open_dataset(era5_path_fine + '/variables/raw/2m_temperature/ERA5_surface.monthly.mean.2m_temperature.1959-01.nc')

### Build the regridder

In [30]:
# ds_out = xr.Dataset(
#     {
#         "lat": (["lat"], ds_1deg.lat.values),
#         "lon": (["lon"], ds_1deg.lon.values),
#     }
# )
ds_out = xe.util.grid_2d(-180.0, 180.0, 1, -90.0, 90.0, 1)


# ds_in = xr.Dataset(
#     {
#         "lat": (["lat"], np.flip(ds_25deg.latitude.values)),
#         "lon": (["lon"], ds_25deg.longitude.values),
#     }
# )

ds_in = xr.Dataset(
    {
        "lat": (["lat"], (ds_25deg.latitude.values)),
        "lon": (["lon"], ds_25deg.longitude.values),
    }
)

In [31]:
ds_out

<xarray.Dataset>
Dimensions:  (y: 180, x: 360, y_b: 181, x_b: 361)
Coordinates:
    lon      (y, x) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
    lat      (y, x) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5 89.5
    lon_b    (y_b, x_b) float64 -180.0 -179.0 -178.0 ... 178.0 179.0 180.0
    lat_b    (y_b, x_b) float64 -90.0 -90.0 -90.0 -90.0 ... 90.0 90.0 90.0 90.0
Dimensions without coordinates: y, x, y_b, x_b
Data variables:
    *empty*

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179])

In [32]:
ds_in


<xarray.Dataset>
Dimensions:  (lat: 721, lon: 1440)
Coordinates:
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
Data variables:
    *empty*

In [47]:
# regrid = xe.Regridder(ds_in, ds_out, method='bilinear')
# regridder = xe.Regridder(ds_in, ds_out, 'bilinear', periodic=True)
regridder = xe.Regridder(ds_in, ds_1deg, 'bilinear', periodic=True)


In [ ]:
for var in varlist:
    
    print(var)
    
    # make folder if it doesn't exist
    new_path = era5_path_coarse + var +'/'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    
    # get list of .25 degree files
    
    filelist = sorted(glob.glob(era5_path_fine + 'variables/raw/'+var+'/*.nc'))
    
    for file in filelist:
        
        coarse_file = new_path +  os.path.basename(file)
        fine_file = file
        
        ds_old_temp = xr.open_dataset(fine_file)
        ds_old = ds_old_temp.rename_dims({'longitude':'lon','latitude':'lat'})
        ds_old = ds_old.assign_coords({'lat':ds_old_temp.latitude.values,
                                       'lon':ds_old_temp.longitude.values})
        
        
        ds_new = regridder(ds_old)
        
        # ds_new.rename_dims({'y':'latitude','x':'longitude'})
        # ds_new.assign_coords({'latitude': ,
        #                       'longitude': })
        
        ds_new.to_netcdf(coarse_file)
    
    
    

2m_dewpoint_temperature
2m_temperature
specific_humidity
relative_humidity
geopotential
surface_net_solar_radiation
surface_latent_heat_flux
mean_sea_level_pressure
leaf_area_index_high_vegetation
leaf_area_index_low_vegetation
medium_cloud_cover
sea_surface_temperature
high_cloud_cover
low_cloud_cover
skin_temperature
u_component_of_wind
v_component_of_wind
temperature


In [ ]:
all_2m_files = sorted(glob.glob('/Volumes/OneTouch/ACDC/ERA5_ACDC_1deg/2m_temperature/*.nc'))

In [ ]:
ds_2m_ts = xr.open_mfdataset(all_2m_files)


In [11]:
all_2m_files = sorted(glob.glob('/Volumes/OneTouch/ACDC/ERA5_ACDC_1deg/2m_temperature/*.nc'))

In [13]:
ds_2m_ts = xr.open_mfdataset(all_2m_files)


In [14]:
ds_2m_ts_short = ds_2m_ts.sel(time=slice('1959-01-01', '1979-12-31'))

In [15]:
ds_2m_ts_short.to_netcdf('/Users/mlague/Documents/ACDC/ACDC2022/ERA5_surface.monthly.mean.2m_temperature.1959-01_to_1979-12.nc')